In [1]:
import pandas as pd
import requests
import json
from dotenv import load_dotenv
import os
import time
import numpy as np

In [2]:
load_dotenv("../../.env",override=True)
consumerKey = os.environ.get("CONSUMER_KEY")


'31PFQ7DIYR4X7XAHQTYTBQJ8NXLBTTCB'

In [3]:
pd.set_option("display.max_columns",500)
pd.set_option("display.max_rows",20)

In [4]:
def request_fundamentals(Tickers: list, consumerKey: str) -> pd.DataFrame:
    """_summary_

    Args:
        Tickers (list)
        consumerKey (str)

    Returns:
        pd.DataFrame
    """
    symbolList = ",".join([i for i in Tickers])
    endpoint = f'''https://api.tdameritrade.com/v1/instruments?
        &symbol={symbolList}&projection=fundamental'''
    page = requests.get(url=endpoint, 
        params={'apikey' : consumerKey})
    content = json.loads(page.content)

    allDict = {}
    for k in list(content.keys()):
        allDict[k] = content[k]['fundamental']
    fundamentals = pd.DataFrame(allDict).T
        
    return fundamentals

def request_quotes(consumerKey: str, Tickers: list) -> pd.DataFrame:
    """_summary_

    Args:
        consumerKey (str): _description_
        Tickers (list): _description_

    Returns:
        pd.DataFrame: _description_
    """
    
    symbolList = ",".join([i for i in Tickers])
    endpoint = f"https://api.tdameritrade.com/v1/marketdata/quotes?symbol={symbolList}"
    page = requests.get(url=endpoint, 
                params={'apikey' : consumerKey})
    content = json.loads(page.content)
    content = pd.DataFrame(content).T
    return content

In [5]:
symbols = pd.read_excel("../../data/static_files/LargeCapFina.xlsx",header=1)
symbols['Symbol'].fillna("-",inplace=True)
Tickers = list(symbols[(~symbols['Symbol'].astype(str).str.contains('-'))]['Symbol'])


c:\Users\jliv\Desktop\research\python_research\.venv\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [6]:

FDF = request_fundamentals(Tickers=Tickers,consumerKey=consumerKey)
Quotes = request_quotes(Tickers=Tickers,consumerKey=consumerKey)

In [7]:
highGood = ['dividendAmount','dividendYield',
    'grossMarginTTM','grossMarginMRQ','netProfitMarginTTM','netProfitMarginMRQ',
    'operatingMarginTTM','operatingMarginMRQ',
    'returnOnEquity', 'returnOnAssets', 'returnOnInvestment',
    'quickRatio','currentRatio','interestCoverage',
    'epsTTM','epsChangePercentTTM','epsChangeYear','epsChange',
    'revChangeYear', 'revChangeTTM','revChangeIn','marketCap',
    'divGrowthRate3Year','dividendPayAmount','beta']

lowGood = [
    'peRatio', 'pegRatio', 'pbRatio', 'prRatio', 'pcfRatio',
    'totalDebtToCapital', 'ltDebtToEquity', 'totalDebtToEquity',
    'shortIntToFloat', 'shortIntDayToCover'
]

In [8]:

FDF=pd.DataFrame(np.where(FDF==0,np.nan,FDF),columns = FDF.columns)


In [9]:
Quotes[['symbol','lastPrice']].merge(FDF[['symbol','peRatio']],
    on='symbol',how='inner')

,symbol,lastPrice,peRatio
0,ACGL,48.175,11.64515
1,AEG,4.1289,20.83586
2,AFG,129.46,10.52406
3,AFL,60.625,8.92309
4,AIG,53.11,3.39561
...,...,...,...
138,WRB,71.22,17.19004
139,WTFC,91.9,12.23961
140,WTW,208.84,14.74719
141,XP,17.675,13.71316


In [10]:
FDF

,symbol,high52,low52,dividendAmount,dividendYield,dividendDate,peRatio,pegRatio,pbRatio,prRatio,pcfRatio,grossMarginTTM,grossMarginMRQ,netProfitMarginTTM,netProfitMarginMRQ,operatingMarginTTM,operatingMarginMRQ,returnOnEquity,returnOnAssets,returnOnInvestment,quickRatio,currentRatio,interestCoverage,totalDebtToCapital,ltDebtToEquity,totalDebtToEquity,epsTTM,epsChangePercentTTM,epsChangeYear,epsChange,revChangeYear,revChangeTTM,revChangeIn,sharesOutstanding,marketCapFloat,marketCap,bookValuePerShare,shortIntToFloat,shortIntDayToCover,divGrowthRate3Year,dividendPayAmount,dividendPayDate,beta,vol1DayAvg,vol10DayAvg,vol3MonthAvg
0,APO,81.07,45.62,1.7,3.46,2022-08-17 00:00:00.000,NaN,NaN,7.62871,5.18987,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.21277,214.9497,214.9497,NaN,NaN,NaN,NaN,NaN,NaN,159.6571,570990539.0,328.8245,28137.25,27.58663,NaN,NaN,NaN,0.4,2022-08-31 00:00:00.000,1.65383,2410150.0,2410151.0,50807080.0
1,LNC,77.57,43.01,1.8,3.77,2022-10-06 00:00:00.000,9.22975,NaN,0.88521,0.41423,NaN,NaN,NaN,4.48179,4.66301,6.92131,7.46473,5.68972,0.24584,NaN,NaN,NaN,5.60294,41.42547,70.72268,70.72268,5.17674,NaN,NaN,NaN,NaN,3.98793,8.89695,170226073.0,154.1203,8133.402,12.33069,NaN,NaN,NaN,0.45,2022-11-01 00:00:00.000,1.7822,1528820.0,1528823.0,37826230.0
2,PNC,228.1425,143.5213,6.0,4.05,2022-10-14 00:00:00.000,11.59028,NaN,1.27633,3.01461,8.69611,NaN,NaN,28.41744,29.24159,34.9692,35.88741,10.51829,1.04493,NaN,NaN,NaN,NaN,NaN,54.52867,54.52867,12.76759,NaN,39.44296,NaN,NaN,16.37601,12.59437,410124004.0,408.7384,60690.15,7.0731,NaN,NaN,NaN,1.5,2022-11-05 00:00:00.000,1.14572,2364940.0,2364944.0,36845660.0
3,BBD,4.737,2.93,0.04,1.16,2022-10-04 00:00:00.000,4.58411,0.144597,NaN,2.03904,NaN,NaN,NaN,22.80075,13.8801,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.84204,31.70266,NaN,NaN,NaN,30.88359,NaN,5320094147.0,9916.555,37632.55,7.37777,NaN,NaN,NaN,0.00527,2022-10-04 00:00:00.000,0.8629,34097080.0,34097080.0,597110000.0
4,RJF,117.37,84.86,1.36,1.39,2022-09-30 00:00:00.000,13.95866,0.556254,2.24904,1.91753,12.91263,96.00435,95.10439,13.58159,10.83513,17.81281,14.9388,17.34848,2.09252,12.12636,NaN,NaN,NaN,84.98268,35.63847,564.5297,7.02503,25.09402,NaN,NaN,NaN,18.66466,2.47141,215824993.0,194.3175,21163.8,11.59357,NaN,NaN,NaN,0.34,2022-10-17 00:00:00.000,1.03066,1363480.0,1363481.0,21440510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,BBVA,7.255,3.93,0.23,4.84,2022-10-07 00:00:00.000,5.76827,4.966396,0.69022,1.15307,3.99344,NaN,NaN,24.0585,18.65598,34.64803,33.04447,11.60764,0.93145,NaN,NaN,NaN,NaN,NaN,129.794,139.1048,0.85294,1.16146,112.0066,NaN,NaN,1.02383,5.65176,6030116564.0,5967.75,29773.62,NaN,NaN,NaN,NaN,0.18476,2022-10-07 00:00:00.000,1.26979,1995360.0,1995360.0,52065200.0
154,FRC,222.86,110.195,1.08,0.97,2022-10-26 00:00:00.000,13.28013,0.846096,1.51261,3.52056,16.08927,NaN,NaN,28.98345,29.21865,36.76927,37.29481,12.04145,0.88931,NaN,NaN,NaN,NaN,NaN,7.47123,7.47123,8.39374,15.69577,15.64863,NaN,NaN,23.32185,15.34954,176742000.0,181.7716,20399.01,14.61956,NaN,NaN,NaN,0.27,2022-11-10 00:00:00.000,1.12685,2202900.0,2202900.0,23568920.0
155,XP,36.68,16.705,NaN,NaN,,13.71316,0.449501,3.00585,3.76204,12.47631,NaN,NaN,28.45524,26.61368,30.86028,27.56285,25.55695,2.59585,3.30951,NaN,NaN,12.14915,72.00893,23.89595,257.3106,1.22948,30.50749,NaN,NaN,NaN,27.99628,9.85642,443803714.0,384.2701,9365.132,NaN,NaN,NaN,NaN,0.0042,2021-09-01 00:00:00.000,NaN,5575650.0,5575653.0,89446910.0
156,AXP,199.55,130.65,2.08,1.46,2022-10-13 00:00:00.000,14.61187,1.099279,4.60329,2.19155,11.63551,67.52663,67.19187,15.6148,14.46032,20.54676,18.72331,30.61057,3.87895,8.17941,NaN,NaN,232.1818,85.66847,174.2845,597.762,9.74687,13.29223,NaN,NaN,NaN,28.17457,14.54837,749747789.0,748.6652,106779.1,5.14347,NaN,NaN,NaN,0.52,2022-11-10 00:00:00.000,1.1386,3379170.0,3379173.0,64317660.0
